# 2 - Processing the data from Google Earth

##### Import

In [ ]:
import sys, mplstereonet
sys.path.append(r"..")  # append local path to access rgeomod module
import rgeomod

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

## 2.1 - Load Google Earth's .kml files

In [ ]:
folder_path = "../data/FW1/"
ks, ks_names, ks_bool = rgeomod.read_kml_files(folder_path)

*ks* contains the KmlPoints objects, containing the point data from the respective .kml files in the given directory:

In [ ]:
ks

*ks_names* contains the filenames:

In [ ]:
ks_names

*ks_bool* is a boolean array specifying which object contains dip values:

In [ ]:
ks_bool

## 2.2 - Get elevation data from Digital Elevation Model (DTM)

In [ ]:
geotiff_filepath = "../data/dome_sub_sub_utm.tif"

In [ ]:
rgeomod.get_elevation_from_dtm(ks, geotiff_filepath)

## 2.3 - Fit planes to point sets

In [ ]:
rgeomod.fit_planes_to_points(ks)

## 2.4 - Convert data into readable DataFrames

In [ ]:
interfaces, foliations = rgeomod.convert_to_df(ks, ks_names, ks_bool)

In [ ]:
interfaces.tail()

In [ ]:
foliations.tail()

## 2.5 - Visualize the data

### 2.5.1 - 3D Point Cloud

In [ ]:
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(interfaces["X"], interfaces["Y"], interfaces["Z"], color="red", alpha=0.85, s=35, label="Interface data")
ax.scatter(foliations["X"], foliations["Y"], foliations["Z"], color="black", alpha=0.85, s=35, label="Foliation data")

ax.legend()
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
fig.suptitle("GoogleEarth picks")

Plot histograms of the extracted dip and dip direction data

### 2.5.2 - Histograms

In [ ]:
fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.hist(foliations["dip"], 18)
ax1.set_xlabel("Dips")
ax1.set_ylabel("Counts")

ax2.hist(foliations["azimuth"], 18, color='r');
ax2.set_xlabel("Dip Directions")
ax2.set_ylabel("Counts");

### 2.5.3 - Stereonet plots

In [ ]:
fig, ax = mplstereonet.subplots()
cax = ax.density_contourf(foliations["azimuth"], foliations["dip"], measurement='poles', cmap='viridis', alpha=0.75)
fig.colorbar(cax)
ax.pole(foliations["azimuth"], foliations["dip"], 'g^', markersize=4)
ax.grid(True, color="black", alpha=0.25)

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='stereonet')
ax.pole(foliations["azimuth"], foliations["dip"], 'g^', markersize=4)
ax.plane(foliations["azimuth"], foliations["dip"], 'g-', linewidth=0.5, alpha=0.85)
ax.grid(True, color="black", alpha=0.25)

## 2.6 - Save data to .csv files

In [ ]:
interfaces.to_csv("../data/gempy_interfaces.csv", index=False)
foliations.to_csv("../data/gempy_foliations.csv", index=False)